In [1]:
# ! pip install treys

In [2]:
import re
import pandas as pd
import numpy as np

from treys import Card
from treys import Evaluator

## Funções de tratamento de dados:

In [3]:
def getPluribusData(): 
    """
    Carrega o arquivo com os dados originais
    """
    pluribus_raw_lines = [] 
    with open('pluribus_entire_raw_data.log', 'rt') as pluribus_file: 
        for file_line in pluribus_file:
            pluribus_raw_lines.append(file_line)
    return pluribus_raw_lines 
        


In [4]:
def getPlays(line):
  """
    A partir da linha inicial de dados do Pluribus,
    separa as informações.
  """
  data = {
          'handIndex' : '',
              'play'  : [],
       'cardsPlayers' : [],
         'cardsTable' : [],
              'bets'  : [],
             'players': []    
  }
  cards = ""

  line = list(line)
  twoPoints = 0
  for currentState in line:
    if (currentState == ':'):
      twoPoints += 1
    elif twoPoints == 1 :
      data['handIndex'] = currentState
    elif twoPoints == 2 :
      data['play'].append(currentState)
    elif twoPoints == 3 :
      cards += currentState
    elif twoPoints == 4 :
      data['bets'].append(currentState)
    elif twoPoints == 5 :
      data['players'].append(currentState)

  #Treating players name
  #data['players'].pop() #Remove '\n'
  if data['players'][-1] == "\n":
    data['players'].pop()
    
  if data['players'][-1] == "E":
    data['players'] = data['players'][:len(data['players'])-5]
  data['players'] = ''.join(data['players']).split('|') #Join the name of the players

  #Treating bets
  data['bets'] = ''.join(data['bets']).split('|') #Join bet values
  data['bets'] = [float(i) for i in data['bets']]

  #Treating Rounds
  data['rounds_quantity'] = data['play'].count('/')
  data['play'] = ''.join(data['play']).split('/')
  

  #Treating Cards:
  if data['rounds_quantity'] == 0:
    data['cardsPlayers'] = cards
  else:
    data['cardsPlayers'], data['cardsTable'] = cards.split('/', 1)

  data['cardsPlayers'] = ''.join(data['cardsPlayers']).split('|')
  data['cardsTable'] = ''.join(data['cardsTable']).split('/')

  return data

In [5]:
import re

def extractRaises(plays):
  
    joinPlays = ''.join(map(str, plays))
    number_ = re.findall("[-\d]+", joinPlays)

    cleanPlays = []
    for play in plays:
        cleanPlays.append(''.join([i for i in play if not i.isdigit()]))

    if len(number_) != 0:
        #Treating raises to be percentages and be a part from the game
        #Other raises:
        for x in range(len(number_)-1, 0, -1):
            #raise_value = int(number_[x]) - int(number_[x-1])
            #percentage = float(raise_value)/float(10000)
            #round_percentage = round(0.025 * round(percentage/0.025) , 3)#rounding for 2.5% to abstract results
            #number_[x] = round_percentage
            raise_value = int(number_[x]) - int(number_[x-1])
            number_[x] = float(raise_value)
            
    
        #the first raise:
        #percentage_firstNumber = float(number_[0])/float(10000)
        #round_firstNumber =round( 0.025 * round(percentage_firstNumber/0.025) , 3)
        #number_[0] = round_firstNumber
        number_[0] = float(number_[0])
    
    return (number_,cleanPlays)

In [6]:

def orderPlays(playsWithRaise):
  """
  Separa as rodadas por jogadores e suas ações, com a porcentagem do raise e o valor no pote
  """

  #Initializing data structure
  #Each inside array represets a game state: preflop, flop...
  data = [[], [], [], []]
  round_counter = -1

  #Treating small and big blinds
  small_blind_value = float(50)
  big_blind_value = float(100)
  data[0].append([0, 0, 'sb', small_blind_value])  #sb stands for smallBlind
  data[0].append([1, small_blind_value, 'bb', big_blind_value])  #bb stands for bigBlind 

  #Managing players on match
  players_on_match = [0,1,2,3,4,5]
  moment_player = 2

  #Managing raise values
  raises, normalPlays = extractRaises(playsWithRaise)
  count_raise = 0
  biggest_raise = 0
  raise_value = 0
  already_payed_raises = [0,0,0,0,0,0]

  #Managing pot value
  pot = small_blind_value + big_blind_value


  for game_round in normalPlays: #Loop through game states
    round_counter += 1

    #In the beginning of every round the raise state is zero
    raise_value = 0 
    biggest_raise = 0
    already_payed_raises = [0,0,0,0,0,0]

    for bet in game_round: #Loop through player bets
      #Looping through players
      if moment_player == len(players_on_match):
        moment_player = 0

      #Call and Checks
      if(bet == 'c'):

        data[round_counter].append([players_on_match[moment_player],
                                    round(pot,3),
                                    'c',
                                    0])
        
        pay = biggest_raise - already_payed_raises[players_on_match[moment_player]]
        
        pot +=  pay 
        already_payed_raises[players_on_match[moment_player]] += pay

        moment_player += 1
        
      #Fold
      if(bet == 'f'):
        data[round_counter].append([players_on_match[moment_player],
                                    round(pot,3),
                                    'f',
                                    0])
        
        del players_on_match[moment_player]

      #Raise
      if(bet == 'r'):
        raise_value = raises[count_raise]
        biggest_raise += raise_value
        count_raise += 1
        
        data[round_counter].append([players_on_match[moment_player],
                                    round(pot,3),
                                    'r',
                                    raise_value])
        
        pay = biggest_raise - already_payed_raises[players_on_match[moment_player]]

        already_payed_raises[players_on_match[moment_player]] += pay
        pot += pay
        moment_player += 1
  return data

In [7]:
# https://github.com/BrenoCPimenta/Poker-preflop-hand-rank-scraping-to-csv
def getPreFlopCSV():
    preflopRank_DF = pd.read_csv('preFlop-rank.csv')
    preflopRank = preflopRank_DF.values.tolist()
    preflopRank.insert(0, preflopRank_DF.columns.tolist())

    return preflopRank

In [8]:
#https://github.com/ihendley/treys

def findPreFlopRank(cards, suit_char, preflopRank):
    """
    Receive pocket cards and returns the preflop rank for them. 
    """
    for index in range(len(preflopRank)):
        if (cards == preflopRank[index][1] or cards[::-1] == preflopRank[index][1]) and suit_char == preflopRank[index][2]:
            rank_before_conversion_range = int(preflopRank[index][0])
            #This multiplication is to acert the range of this rank to be the same as the trey's librarie:
            rank_after_conversion_range = ((rank_before_conversion_range - 1) * 44 ) + 1
            return rank_after_conversion_range
            


def setHandsStrenght(data, cards_players, cards_table, preflopRank):
  #Setting Board cards
  table_cards = []
  if cards_table[0] != '':
    for rounds in range(len(cards_table)):
      if rounds == 0:
        table_cards.append([Card.new(cards_table[rounds][:2]),
                            Card.new(cards_table[rounds][2:4]),
                            Card.new(cards_table[rounds][4:6])
                            ])
      if rounds == 1:
        table_cards.append((table_cards[0]).copy())
        table_cards[1].append(Card.new(cards_table[rounds]))
      
      if rounds == 2:
        table_cards.append(table_cards[1].copy())
        table_cards[2].append(Card.new(cards_table[rounds]))
      

  #Setting hand strenght into data
  evaluator = Evaluator()

  for game_state in range(len(data)):
    if 1 ==1: #data[game_state] != []:
      for hand in range(len(data[game_state])):
        hand_cards = cards_players[data[game_state][hand][0]]

        if game_state == 0:
          if hand_cards[1] == hand_cards[3]:
            suit_char = 's'
          else:
            suit_char = 'o'
          
          if hand_cards[0] == hand_cards[2]:
            suit_char = 'p'

          only_cards = hand_cards[0] + hand_cards[2]
          hand_strength = findPreFlopRank(only_cards, suit_char, preflopRank)
          
        else:
          if len(table_cards) != 0:
            hand_cards_obj = [Card.new(hand_cards[:2]),
                              Card.new(hand_cards[2:])                          
            ]
            hand_strength = evaluator.evaluate(table_cards[game_state - 1], hand_cards_obj)

        data[game_state][hand].insert(2,hand_strength)

  return data


In [9]:
def linearizeData(raw_data):
    data = []
    situation_name = ''

    for situation in range(len(raw_data)):
        #In order to keep numerical values this block is drepecated
        '''
        if situation == 0: situation_name = "preflop" 
        if situation == 1: situation_name = "flop" 
        if situation == 2: situation_name = "turn" 
        if situation == 3: situation_name = "river"
        '''
        turns_per_players = [0, 0, 0, 0, 0, 0]

        for turn in range(len(raw_data[situation])):
            players_turn = turns_per_players[raw_data[situation][turn][0]]
            
            turns_per_players[raw_data[situation][turn][0]] += 1

            raw_data[situation][turn].insert(1,situation)
            raw_data[situation][turn].insert(2,players_turn)

            data.append(raw_data[situation][turn])

    return data

def linearizedDataToDataFrame(data):
    df = pd.DataFrame(np.array(data),
                        columns=['Position',
                                  'Situation',
                                  'Turn',
                                  'Pot',
                                  'Hand',
                                  'Action',
                                  'Raise'])
    return df

In [10]:
def zero_games(games):
  for it in range(len(games)):
    # print(games[it][0])

    games[it][1] = 0
    if games[it][0] != 'f' and games[it][0] != 'out':
      games[it][0] = 'hold'
    else:
      games[it][0] = 'out'
  
  return games

def treatingFolds(games, player, last_player):
  # print("Entrou treating Folds: Last_Player:", last_player, " e player:", player)
  if player < last_player:
    last_player = last_player-6
    
  distance = player - last_player
  
  if distance == 1:
    return games
  else:
    while(distance != 1):
      # print("Entrou while com distance:", distance)
      distance -= 1

      last_player += 1

      if last_player > 5:
        last_player = 0
      # print("Player zerado é o ", player)
      games[last_player][0] = "out"
      games[last_player][1] = 0
    return games
      
      

def getGameState(data):
  games = [['sb', 50],
          ['bb', 100],
          ['hold', 0],
          ['hold', 0],
          ['hold', 0],
          ['hold', 0]]
  last_player = 1
  last_situation = data[0][1]

  for it in range(len(data)):
    # print(0)
    if(it != 0 and it != 1):
      current_situation = data[it][1]

      if current_situation != last_situation:
        games = zero_games(games)
        last_situation = current_situation

      player = data[it][0]    
      action = data[it][5]
      bet = data[it][6]

      games[player][0] = action
      games[player][1] = bet

      if current_situation == last_situation:
        games = treatingFolds(games, player, last_player)

      for i in range(6):
        if i == player:
          pass
        else:
          data[it].append(games[i][0])
          data[it].append(games[i][1])

      last_player = player

  
  return data


def gameStateToDataFrame(gameState):
  test_df = pd.DataFrame(np.array(gameState[2:]),
                          columns=['Position',
                                    'Situation',
                                    'Turn',
                                    'Pot',
                                    'Hand',
                                    'Action',
                                    'Raise',
                                    'P1-Action',
                                    'P1-Bet',
                                    'P2-Action',
                                    'P2-Bet',
                                    'P3-Action',
                                    'P3-Bet',
                                    'P4-Action',
                                    'P4-Bet',
                                    'P5-Action',
                                    'P5-Bet'])
            
  return test_df

In [11]:
def tratamento_completo_linha(linha, preflopRank):
    game = getPlays(linha) 
    hands = setHandsStrenght(orderPlays(game['play']),
                            game['cardsPlayers'],
                            game['cardsTable'],
                            preflopRank)
    linear_hand = linearizeData(hands)
    gameState = getGameState(linear_hand)
    df_gameState = gameStateToDataFrame(gameState)
    dict_gameState = df_gameState.to_dict()
    return dict_gameState

----

# Visualização parcial do tratamento:

Escolha a linha que será mostrado o tratamento passo a passo:

In [12]:
linha = 1362

* **Dado Original:**

In [13]:
pluribus_raw_lines  = getPluribusData()
print("Linha do dataset original: \n\n", pluribus_raw_lines[linha])

Linha do dataset original: 

 STATE:2:ffffr250f:9cTh|6cJs|5d7h|5s9s|4d6s|Jd3h:100|-100|0|0|0|0:MrBlonde|Eddie|Bill|Pluribus|MrOrange|MrWhite



* **Separação das informações:**

In [14]:
game = getPlays(pluribus_raw_lines[linha]) 
print("Jogadores             ", game['players'])
print()
print("Ações:                ", game['play'])
print()
print("Valores ganhos:       ", game['bets'])
print()
print("Cartas dos jogadores: ", game['cardsPlayers'])
print() 
print("Cartas da mesa:       ", game['cardsTable'])  
game = getPlays(pluribus_raw_lines[linha])

Jogadores              ['MrBlonde', 'Eddie', 'Bill', 'Pluribus', 'MrOrange', 'MrWhite']

Ações:                 ['ffffr250f']

Valores ganhos:        [100.0, -100.0, 0.0, 0.0, 0.0, 0.0]

Cartas dos jogadores:  ['9cTh', '6cJs', '5d7h', '5s9s', '4d6s', 'Jd3h']

Cartas da mesa:        ['']


* **Extraindo valor das apostas:**

In [15]:
print("Apostas separadas das jogadas: ", extractRaises(game['play']))

Apostas separadas das jogadas:  ([250.0], ['ffffrf'])


* **Organizando as jogadas:**

In [16]:
st_cnt = -1
for situation in orderPlays(game['play']):
    st_cnt += 1
    print("---Situação do jogo ", st_cnt)
    for turn in situation:
      print(turn)

---Situação do jogo  0
[0, 0, 'sb', 50.0]
[1, 50.0, 'bb', 100.0]
[2, 150.0, 'f', 0]
[3, 150.0, 'f', 0]
[4, 150.0, 'f', 0]
[5, 150.0, 'f', 0]
[0, 150.0, 'r', 250.0]
[1, 400.0, 'f', 0]
---Situação do jogo  1
---Situação do jogo  2
---Situação do jogo  3


* **Criando um valor abstrato de força para cada mão:**

In [17]:
preflopRank = getPreFlopCSV()
hands = setHandsStrenght(orderPlays(game['play']),
                  game['cardsPlayers'],
                  game['cardsTable'],
                  preflopRank)
cnt = -1
for rounds in hands:
    cnt+=1
    print("Round "+str(cnt))
    for hand in rounds:
      print(hand)

Round 0
[0, 0, 3169, 'sb', 50.0]
[1, 50.0, 6425, 'bb', 100.0]
[2, 150.0, 5677, 'f', 0]
[3, 150.0, 3829, 'f', 0]
[4, 150.0, 5941, 'f', 0]
[5, 150.0, 6689, 'f', 0]
[0, 150.0, 3169, 'r', 250.0]
[1, 400.0, 6425, 'f', 0]
Round 1
Round 2
Round 3


* **Linearizando dados e os tornando mais legíveis:**

In [18]:
linear_hand = linearizeData(hands)

In [19]:
linearizedDataToDataFrame(linear_hand)

,Position,Situation,Turn,Pot,Hand,Action,Raise
0,0,0,0,0,3169,sb,50.0
1,1,0,0,50.0,6425,bb,100.0
2,2,0,0,150.0,5677,f,0
3,3,0,0,150.0,3829,f,0
4,4,0,0,150.0,5941,f,0
5,5,0,0,150.0,6689,f,0
6,0,0,1,150.0,3169,r,250.0
7,1,0,1,400.0,6425,f,0


* **Juntando a cada jogada, as últimas ações dos outros jogadores:**

In [20]:
gameState = getGameState(linear_hand)

In [21]:
gameStateToDataFrame(gameState)

,Position,Situation,Turn,Pot,Hand,Action,Raise,P1-Action,P1-Bet,P2-Action,P2-Bet,P3-Action,P3-Bet,P4-Action,P4-Bet,P5-Action,P5-Bet
0,2,0,0,150.0,5677,f,0,sb,50,bb,100,hold,0,hold,0,hold,0
1,3,0,0,150.0,3829,f,0,sb,50,bb,100,f,0,hold,0,hold,0
2,4,0,0,150.0,5941,f,0,sb,50,bb,100,f,0,f,0,hold,0
3,5,0,0,150.0,6689,f,0,sb,50,bb,100,f,0,f,0,f,0
4,0,0,1,150.0,3169,r,250.0,bb,100,f,0,f,0,f,0,f,0
5,1,0,1,400.0,6425,f,0,r,250.0,f,0,f,0,f,0,f,0


Após todas as funções a cima temos os dados tratados.

## Separação de dados para os modelos

As nossas previsões só ocorrerão no primeiro turno do PreFlop, ou seja onde Situation e Turn são zeros. Essa decisão foi tomada, pois poker é um jogo muito complexo e a parte mais simples/mecânica, onde há mais padrões, acontece no início do jogo, onde não temos cartas na mesa e não precisamos analisar posturas de jogadas de turnos anterioes.
<br>
Dentre esses dados separaremos um dataset para Regressão do valor de aposta e outro para a classificação da Ação. 

In [22]:
class_data = {
    'Action' : [],
    'Position' : [],
    'Hand' : [],
    'Pot' : [],
    'P1-Action' : [],
    'P1-Bet' : [],
    'P2-Action' : [],
    'P2-Bet' : [],
    'P3-Action' : [],
    'P3-Bet' : [],
    'P4-Action' : [],
    'P4-Bet' : [],
    'P5-Action' : [],  
    'P5-Bet' : []
}

regress_data = {
    'Raise' : [],
    'Position' : [],
    'Hand' : [],
    'Pot' : [],
    'P1-Action' : [],
    'P1-Bet' : [],
    'P2-Action' : [],
    'P2-Bet' : [],
    'P3-Action' : [],
    'P3-Bet' : [],
    'P4-Action' : [],
    'P4-Bet' : [],
    'P5-Action' : [],  
    'P5-Bet' : []
}

In [23]:
for linha in pluribus_raw_lines:
    jogadas = tratamento_completo_linha(linha, preflopRank)
    for i in range(len(jogadas['Action'])):
        if( ( float(jogadas['Situation'][i]) == 0 and float(jogadas['Turn'][i]) == 0 ) 
            or
            (                                                                         
              ( float(jogadas['Situation'][i]) == 0 and float(jogadas['Turn'][i]) == 1 )
                 and
              ( float(jogadas['Position'][i]) == 0 or float(jogadas['Position'][i]) == 1 )
            )
          ):
            class_data['Action'].append(jogadas['Action'][i])
            class_data['Position'].append(int(jogadas['Position'][i]))
            class_data['Hand'].append(int(jogadas['Hand'][i]))
            class_data['Pot'].append(float(jogadas['Pot'][i]))
            class_data['P1-Action'].append(jogadas['P1-Action'][i])
            class_data['P2-Action'].append(jogadas['P2-Action'][i])
            class_data['P3-Action'].append(jogadas['P3-Action'][i])
            class_data['P4-Action'].append(jogadas['P4-Action'][i])
            class_data['P5-Action'].append(jogadas['P5-Action'][i])
            class_data['P1-Bet'].append(float(jogadas['P1-Bet'][i]))
            class_data['P2-Bet'].append(float(jogadas['P2-Bet'][i]))
            class_data['P3-Bet'].append(float(jogadas['P3-Bet'][i]))
            class_data['P4-Bet'].append(float(jogadas['P4-Bet'][i]))
            class_data['P5-Bet'].append(float(jogadas['P5-Bet'][i]))

            if jogadas['Action'][i] == 'r':
                regress_data['Raise'].append(float(jogadas['Raise'][i]))
                regress_data['Position'].append(int(jogadas['Position'][i]))
                regress_data['Hand'].append(int(jogadas['Hand'][i]))
                regress_data['Pot'].append(float(jogadas['Pot'][i]))
                regress_data['P1-Action'].append(jogadas['P1-Action'][i])
                regress_data['P2-Action'].append(jogadas['P2-Action'][i])
                regress_data['P3-Action'].append(jogadas['P3-Action'][i])
                regress_data['P4-Action'].append(jogadas['P4-Action'][i])
                regress_data['P5-Action'].append(jogadas['P5-Action'][i])
                regress_data['P1-Bet'].append(float(jogadas['P1-Bet'][i]))
                regress_data['P2-Bet'].append(float(jogadas['P2-Bet'][i]))
                regress_data['P3-Bet'].append(float(jogadas['P3-Bet'][i]))
                regress_data['P4-Bet'].append(float(jogadas['P4-Bet'][i]))
                regress_data['P5-Bet'].append(float(jogadas['P5-Bet'][i]))
    

#### Salva os datasets para serem usados nos modelos

In [24]:
'''
pd.DataFrame(class_data).to_csv('poker_dados_clasf.csv', encoding='utf-8', index=False)
pd.DataFrame(regress_data).to_csv('poker_dados_regress.csv', encoding='utf-8', index=False)
'''

"\npd.DataFrame(class_data).to_csv('poker_dados_clasf.csv', encoding='utf-8', index=False)\npd.DataFrame(regress_data).to_csv('poker_dados_regress.csv', encoding='utf-8', index=False)\n"